In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

# **Reading Data**

In [3]:
data = pd.read_csv("/content/drive/MyDrive/TextEmotion NLP/train.txt", sep=';')
data.columns = ["Text", "Emotions"]
print(data.head())

                                                Text Emotions
0  i can go from feeling so hopeless to so damned...  sadness
1   im grabbing a minute to post i feel greedy wrong    anger
2  i am ever feeling nostalgic about the fireplac...     love
3                               i am feeling grouchy    anger
4  ive been feeling a little burdened lately wasn...  sadness


# **Tokenizing**

In [4]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

# **Padding the sequences to the same length**

In [6]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

# **Label encoding**

In [7]:
# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# **One-hot encode the labels**

In [8]:
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

# **Text Emotions Classification Model**

## **split the data into training and test sets**

In [9]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences,
                                                one_hot_labels,
                                                test_size=0.2)

## **Model**

In [10]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/10
400/400 [==============================] - 22s 47ms/step - loss: 1.3438 - accuracy: 0.4859 - val_loss: 0.7470 - val_accuracy: 0.7434
Epoch 2/10
400/400 [==============================] - 25s 61ms/step - loss: 0.3035 - accuracy: 0.9076 - val_loss: 0.4963 - val_accuracy: 0.8281
Epoch 3/10
400/400 [==============================] - 19s 47ms/step - loss: 0.0550 - accuracy: 0.9864 - val_loss: 0.5032 - val_accuracy: 0.8338
Epoch 4/10
400/400 [==============================] - 19s 46ms/step - loss: 0.0259 - accuracy: 0.9941 - val_loss: 0.5485 - val_accuracy: 0.8347
Epoch 5/10
400/400 [==============================] - 18s 44ms/step - loss: 0.0179 - accuracy: 0.9964 - val_loss: 0.5828 - val_accuracy: 0.8288
Epoch 6/10
400/400 [==============================] - 18s 45ms/step - loss: 0.0152 - accuracy: 0.9970 - val_loss: 0.6074 - val_accuracy: 0.8297
Epoch 7/10
400/400 [==============================] - 19s 48ms/step - loss: 0.0132 - accuracy: 0.9968 - val_loss: 0.6285 - val_accuracy:

## **Checking the model performance**

In [13]:
# Define a list of input sentences with different emotions
input_sentences = [
    "I'm so excited for the concert tonight!",
    "Feeling sad about the news of my friend's illness.",
    "Happiness overwhelms me as I spend time with loved ones.",
    "The sound of footsteps echoing in the empty hallway sent shivers down my spine."
]

# Initialize an empty list to store predicted labels
predicted_labels = []

# Loop through each input sentence
for input_text in input_sentences:
    # Preprocess the input text
    input_sequence = tokenizer.texts_to_sequences([input_text])
    padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)

    # Make prediction
    prediction = model.predict(padded_input_sequence)

    # Decode the predicted label
    predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])

    # Append the predicted label to the list of predicted labels
    predicted_labels.append(predicted_label)

# Print the predicted labels for each input sentence
for i, predicted_label in enumerate(predicted_labels):
    print(f"Sentence {i+1}: {predicted_label}")


1/1 [==============================] - 0s 24ms/step
Sentence 1: ['joy']
Sentence 2: ['sadness']
Sentence 3: ['love']
Sentence 4: ['sadness']
